### Импорт библиотек

In [ ]:
import random

### Генерация сырых данных

In [ ]:
full_names_gen = [
    "Семёнов Дмитрий Иванович",
    "Головин Сергей Вальерьевич",
    "Путилов Андрей Маркович",
    "Лапухов Алексей Дмитриевич",
    "Дружинин Георгий Михайлович"
]

status_gen = [
    ""
]

print(full_names_gen)

['Семёнов Дмитрий Иванович', 'Головин Сергей Вальерьевич', 'Путилов Андрей Маркович', 'Лапухов Алексей Дмитриевич', 'Дружинин Георгий Михайлович']
